## Simple RAG에서 Chunk 크기 평가하기

Retrieval-Augmented Generation (RAG) 파이프라인에서 **적절한 Chunk 크기를 설정하는 것**은 검색 정확도를 높이는 데 매우 중요합니다. 핵심은 **검색 성능과 응답 품질 사이의 균형**을 맞추는 것입니다.

이 섹션에서는 다양한 Chunk 크기에 대해 아래와 같은 절차로 평가를 진행합니다:

1. PDF에서 텍스트를 추출합니다.
2. 텍스트를 여러 크기의 Chunk로 나눕니다.
3. 각 Chunk에 대해 Embedding을 생성합니다.
4. 쿼리에 대해 관련 있는 Chunk들을 검색합니다.
5. 검색된 Chunk를 바탕으로 응답을 생성합니다.
6. 생성된 응답의 \*\*사실성(faithfulness)\*\*과 \*\*관련성(relevancy)\*\*을 평가합니다.
7. 서로 다른 Chunk 크기에서의 결과를 비교합니다.

## 환경 설정하기
필요한 라이브러리를 가져오는 것부터 시작합니다.

In [2]:
import os
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import json

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('api_key')

## OpenAI API 클라이언트 설정하기

In [3]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)


## PDF 파일에서 텍스트 추출하기

In [ ]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [5]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

txt_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

extracted_text = load_text_file(txt_path)
print(extracted_text[:500])

2023 핵심 개정세법
01
2024 달라지는 세금제도
(국민·기업 납세자용)

[유의사항]
'2023 핵심 개정세법'은 국회에서 의결된 세법 개정사항을 모두 포괄하고 있으나, 시행령·
시행규칙의 경우 2023.7월 발표한 ‘2023 세법개정안'을 중심으로 개정세법과 관련된 내용의
경우 그대로 반영하였습니다. 그러므로 정부의 시행령, 시행규칙 개정 과정에서 일부 변동될
수 있고 새로이 추가 제정될 수도 있습니다. 아울러 실무상 적용할 때는 반드시 개정세법의
구체적인 조문을 확인하셔야 합니다.

국민·기업 납세자용
2024 달라지는 세금제도
2024 달라지는 세금제도
2023 세목별 핵심 개정세법
2023 개정세법 현행-개정사항 비교
01
부동산 세금제도
●(조특법) 연 단위 양도세 감면한도 악용방지를 위해 감면한도 산정방법 조정
양도소득세 산정 및 감면이 연단위로 이뤄지는 점을 감안하여 ▲토지의 일부를
양도한 날부터 소급하여 1년 내 토지를 분할한 경우 분필한 토지 또는 토지 지분



## 추출된 텍스트 청크 분할
검색을 개선하기 위해 추출된 텍스트를 서로 다른 크기의 겹치는 청크로 분할했습니다.

In [6]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 겹치는 n개의 문자 세그먼트로 청크합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 빈 리스트를 초기화합니다.
    
    # (n - overlap) 단계로 텍스트를 반복합니다.
    for i in range(0, len(text), n - overlap):
        # 인덱스 i부터 i + n까지의 텍스트를 청크 리스트에 추가합니다.
        chunks.append(text[i:i + n])

    return chunks  

# 평가할 청크 크기를 정의합니다.
chunk_sizes = [256, 512, 1024]

# 각 청크 크기에 대한 텍스트 청크를 저장할 딕셔너리를 생성합니다.
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# Print the number of chunks created for each chunk size
for size, chunks in text_chunks_dict.items():
    print(f"Chunk Size: {size}, Number of Chunks: {len(chunks)}")

Chunk Size: 256, Number of Chunks: 135
Chunk Size: 512, Number of Chunks: 68
Chunk Size: 1024, Number of Chunks: 34


## 분할된 청크의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [7]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 지정된 텍스트에 대한 임베딩을 생성합니다.

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 인코딩을 위한 배치 크기입니다.

    Returns:
        np.ndarray: 모델에 의해 생성된 임베딩입니다.
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위한 진행 표시줄을 표시합니다.
    )
    
    return embeddings

# GPU 사용 가능 여부를 확인합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델을 로드합니다.
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)

# 각 청크 크기에 대한 임베딩을 생성합니다.
chunk_embeddings_dict = {size: create_embeddings(embedding_model, chunks) for size, chunks in tqdm(text_chunks_dict.items(), desc="Generating Embeddings")}

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating Embeddings: 100%|██████████| 3/3 [00:03<00:00,  1.33s/it]


## 시맨틱 검색 수행하기
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [8]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [9]:
def retrieve_relevant_chunks(query, text_chunks, chunk_embeddings, k=5):
    """
    tok-k의 가장 유사한 청크를 검색
    
    Args:
    query (str): 사용자 쿼리
    text_chunks (List[str]): 텍스트 청크 리스트
    chunk_embeddings (List[np.ndarray]): 텍스트 청크 임베딩 리스트
    k (int): 상위 k개의 관련 텍스트 청크를 반환합니다. 기본값은 5입니다.
    
    Returns:
    List[str]: 가장 유사한 상위 k개의 텍스트 청크 리스트
    """
    # 쿼리에 대한 임베딩을 생성합니다.
    query_embedding = create_embeddings(embedding_model, [query])[0]
    
    # 쿼리 임베딩과 각 청크 임베딩 간의 코사인 유사도를 계산합니다.
    similarities = [cosine_similarity(query_embedding, emb) for emb in chunk_embeddings]
    
    # 가장 유사한 상위 k개의 청크의 인덱스를 추출합니다.
    top_indices = np.argsort(similarities)[-k:][::-1]
    
    # 가장 유사한 상위 k개의 관련 텍스트 청크를 반환합니다.
    return [text_chunks[i] for i in top_indices]

In [10]:
import pandas as pd
# 평가 데이터 로드하기
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

,query,generation_gt
0,피고의 고지의무 위반으로 인해 원고들이 입은 손해는 무엇으로 정의됩니까?,"피고의 고지의무 위반으로 인해 원고들이 입은 손해는 ""이 사건 각 분양계약에 따른 ..."
1,지방공무원법 제53조는 공무원이 직무와 관련하여 무엇을 주거나 받을 수 없다고 규정...,"지방공무원법 제53조는 공무원이 직무와 관련하여 직접적이든 간접적이든 사례, 증여 ..."
2,연금소득의 분리과세 기준금액은 얼마로 인상되었습니까?,연금소득의 분리과세 기준금액은 1천200만원에서 1천500만원으로 인상되었습니다.
3,공무원연금법 제65조 제1항 제1호에 의한 퇴직연금 등의 감액 요건은 무엇입니까?,공무원연금법 제65조 제1항 제1호에 의한 퇴직연금 등의 감액 요건은 재직 중 직무...
4,권리사용료 산출방법에 대한 고시는 언제 개정되었습니까?,권리사용료 산출방법에 대한 고시는 2021년 3월 30일에 개정되었습니다.


In [11]:
# 평가 데이터에서 첫 번째 쿼리를 추출합니다.
query = df['query'][0]

# 각 청크 크기에 대해 관련 청크를 검색합니다.
retrieved_chunks_dict = {size: retrieve_relevant_chunks(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# 청크 크기 256에 대한 검색된 청크를 출력합니다.
print(retrieved_chunks_dict[256])

Batches: 100%|██████████| 1/1 [00:00<00:00, 48.04it/s]

['보호를 위해 비거주자·외국법인에 대한 조세조약상 비과세·면제 경\n15\n\n한국세무사회\n정청구 기한이 원천징수된 날이 속하는 달의 말일부터 5년 이내였던 것을 원천징\n수세액 납부일의 다음 달 10일 이후부터 5년 이내 경정청구할 수 있도록 개선했다.\n● (소득세법 시행령) 임직원의 국외주식 기준 보상 거래내역 제출의무 부여\n내국법인이나 외국법인의 국내사업장의 임직원이 국외지배 주주인 외국법인으로부\n터 받은 주식기준 보상 등을 행사하거나 지급받을 경우 주식 기', ' 전 소유권자의 체납세금이 있는지와 체납세금이 있\n다면 법정기일이 저당권 등기일보다 빠른지를 파악해야 피해를 보지 않을 수 있다.\n●(국세기본법) 상속재산 평가방법 차이에 대한 신고·납부지연가산세 적용 모두 제외\n상속·증여재산 및 부담부 증여자산의 신고 시 해당 자산의 평가 차이로 인한 금\n액에 대해서는 과소신고가산세는 명시 규정이 있으나 납부지연가산세에 대해서는\n9\n\n한국세무사회\n언급이 없어 과세될 수 있었으나, 과소신고가산세는 물론 납부지연가산세도 부', '과세표준을 신고하는 경우부터 적용)\n●(국세기본법) 고발이나 통고처분과 관계없는 세목·세액은 과세전적부심사권 보장\n과세전적부심사 적용제외 사유인 「조세범 처벌법」위반으로 고발 또는 통고처분\n하는 상황에 해당하더라도, 고발 또는 통고처분과 관계없는 세목 또는 세액은 과세\n전적부심사를 할 수 있도록 해 납세자의 권리보호를 강화했다.\n●(국세징수법) 압류금지재산 또는 제3자 재산을 압류한 경우 압류 즉시 해제\n압류금지재산을 압류하거나 제3자의 재산을 압류한 경', '부터 적용)\n●(소득세법) 양도세 이월과세시 증여자 지출분 자본적 지출액도 필요경비로 공제\n배우자 등으로부터 증여받은 후 증여받은 날로부터 10년 이내 양도한 경우. 양\n도소득세 이월과세하는 경우 필요경비로 공제되는 자본적 지출액을 양도자가 지출\n한 것뿐만 아니라 증여자가 지출한 자본적 지출액도 추가로 필요경비 공제를 받을\n수 있도록 하였다

## 검색된 청크를 기반으로 응답 생성하기
청크 크기 `256`에 대해 검색된 텍스트를 기반으로 응답을 생성해 보겠습니다.

In [12]:
def generate_response(system_prompt, query, retrieved_chunks, model_name='gpt-4.1-nona'):
    # Combine retrieved chunks into a single context string
    context = "\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # Create the user prompt by combining the context and the query
    user_prompt = f"{context}\n\nQuestion: {query}"

    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 시스템 프롬프트를 정의합니다.
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
# 응답 생성
# Generate AI responses for each chunk size
ai_responses_dict = {size: generate_response(system_prompt, query,  retrieved_chunks_dict[size],'gpt-4.1-nano-2025-04-14') for size in chunk_sizes}

# Print the response for chunk size 256
print(ai_responses_dict[256])

ChatCompletion(id='chatcmpl-Bmy6gyhK7OeEgtIBt3sokPEJpmfre', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='해당 질문에 답변할 충분한 정보가 없습니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1751010494, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_38343a2f8f', usage=CompletionUsage(completion_tokens=13, prompt_tokens=981, total_tokens=994, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [16]:
print(ai_responses_dict[256].choices[0].message.content)
print(ai_responses_dict[512].choices[0].message.content)

해당 질문에 답변할 충분한 정보가 없습니다.
해당 질문에 답변할 충분한 정보가 없습니다.


## 생성 응답 평가하기
생서 응답을 예상 답변과 비교하여 점수를 부여합니다.

In [17]:
# Define evaluation scoring system constants
SCORE_FULL = 1.0     # Complete match or fully satisfactory
SCORE_PARTIAL = 0.5  # Partial match or somewhat satisfactory
SCORE_NONE = 0.0     # No match or unsatisfactory

In [18]:
# Define strict evaluation prompt templates
FAITHFULNESS_PROMPT_TEMPLATE = """
Evaluate the faithfulness of the AI response compared to the true answer.
User Query: {question}
AI Response: {response}
True Answer: {true_answer}

Faithfulness measures how well the AI response aligns with facts in the true answer, without hallucinations.

INSTRUCTIONS:
- Score STRICTLY using only these values:
    * {full} = Completely faithful, no contradictions with true answer
    * {partial} = Partially faithful, minor contradictions
    * {none} = Not faithful, major contradictions or hallucinations
- Return ONLY the numerical score ({full}, {partial}, or {none}) with no explanation or additional text.
"""

In [19]:
RELEVANCY_PROMPT_TEMPLATE = """
Evaluate the relevancy of the AI response to the user query.
User Query: {question}
AI Response: {response}

Relevancy measures how well the response addresses the user's question.

INSTRUCTIONS:
- Score STRICTLY using only these values:
    * {full} = Completely relevant, directly addresses the query
    * {partial} = Partially relevant, addresses some aspects
    * {none} = Not relevant, fails to address the query
- Return ONLY the numerical score ({full}, {partial}, or {none}) with no explanation or additional text.
"""

In [22]:
def evaluate_response(question, response, true_answer, model='gpt-4.1-mini'):
        """
        faithfulness와 relevancy을 기준으로 AI가 생성한 응답의 품질을 평가합니다.

        Args:
        question (str): 사용자의 원래 질문
        response (str): 평가될 AI 생성 응답
        true_answer (str): 정답으로 사용된 실제 답변

        Returns:
        Tuple[float, float]: (faithfulness_score, relevancy_score)
        """
        # Format the evaluation prompts
        faithfulness_prompt = FAITHFULNESS_PROMPT_TEMPLATE.format(
                question=question, 
                response=response, 
                true_answer=true_answer,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )
        
        relevancy_prompt = RELEVANCY_PROMPT_TEMPLATE.format(
                question=question, 
                response=response,
                full=SCORE_FULL,
                partial=SCORE_PARTIAL,
                none=SCORE_NONE
        )
        faithfulness_response = client_openai.chat.completions.create(
        model=model,
        messages=[
                {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
                {"role": "user", "content": faithfulness_prompt}
                ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        )
        relevancy_response = client_openai.chat.completions.create(
        model=model,
        messages=[
                {"role": "system", "content": "You are an objective evaluator. Return ONLY the numerical score."},
                {"role": "user", "content": relevancy_prompt}
                ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
        )

        
        # 점수를 추출하고 파싱 오류 처리
        try:
                faithfulness_score = float(faithfulness_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: Could not parse faithfulness score, defaulting to 0")
                faithfulness_score = 0.0
                
        try:
                relevancy_score = float(relevancy_response.choices[0].message.content.strip())
        except ValueError:
                print("Warning: Could not parse relevancy score, defaulting to 0")
                relevancy_score = 0.0

        return faithfulness_score, relevancy_score

# 첫 번째 평가 데이터의 정답
true_answer = df['generation_gt'][0]

# 청크 크기 256과 512에 대한 응답 평가
faithfulness, relevancy = evaluate_response(query, ai_responses_dict[256], true_answer, 'gpt-4.1-mini')
faithfulness2, relevancy2 = evaluate_response(query, ai_responses_dict[512], true_answer, 'gpt-4.1-mini')

# 평가 점수 출력
print(f"Faithfulness Score (Chunk Size 256): {faithfulness}")
print(f"Relevancy Score (Chunk Size 256): {relevancy}")

print(f"\n")

print(f"Faithfulness Score (Chunk Size 512): {faithfulness2}")
print(f"Relevancy Score (Chunk Size 512): {relevancy2}")

Faithfulness Score (Chunk Size 256): 0.0
Relevancy Score (Chunk Size 256): 0.0


Faithfulness Score (Chunk Size 512): 0.0
Relevancy Score (Chunk Size 512): 0.0
